# Grid Trading Strategy Example

This notebook demonstrates how to use the grid trading strategy implementation.

## What is Grid Trading?

Grid trading is a strategy that places buy and sell orders at regular price intervals (grids). The strategy profits from price oscillations in ranging markets by:
- Buying at lower grid levels
- Selling at upper grid levels
- Taking profit when price moves between grids

Grid trading works best in **sideways/ranging markets** where price oscillates within a predictable range.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import grid_sim
import grid_result
import instrument
import utils

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Test Grid Strategy on a Single Pair

Let's start by testing the grid strategy on a single currency pair (EUR_USD).

In [ ]:
# Load instrument and price data
pairname = "EUR_USD"
granularity = "H1"

i_pair = instrument.Instrument.get_instrument_by_name(pairname)
price_data = grid_sim.get_price_data(pairname, granularity)

print(f"Loaded {len(price_data)} price bars for {pairname}")
print(f"Price range: {price_data.mid_c.min():.5f} - {price_data.mid_c.max():.5f}")
price_data.head()

In [ ]:
# Test with specific grid parameters
grid_size_pips = 20  # 20 pips between each grid level
num_levels = 10      # 10 levels above and below center

result = grid_sim.evaluate_pair(i_pair, grid_size_pips, num_levels, price_data)

# Print summary
result.print_summary()

## 2. View Individual Trades

In [ ]:
# Get all trades
trades_df = result.get_trades_df()

print(f"Total trades: {len(trades_df)}")
trades_df.head(10)

In [ ]:
# View winning trades
winning_trades = result.get_winning_trades()
print(f"\nWinning trades: {len(winning_trades)}")
print(f"Total profit: {winning_trades.GAIN.sum():.2f} pips")
winning_trades.head()

In [ ]:
# View losing trades
losing_trades = result.get_losing_trades()
print(f"\nLosing trades: {len(losing_trades)}")
print(f"Total loss: {losing_trades.GAIN.sum():.2f} pips")
losing_trades.head()

## 3. Visualize Cumulative Returns

In [ ]:
# Plot cumulative returns
cumulative_returns = result.get_cumulative_returns()

plt.figure(figsize=(12, 6))
plt.plot(cumulative_returns.values)
plt.title(f'Cumulative Returns - {pairname} (Grid: {grid_size_pips} pips, {num_levels} levels)')
plt.xlabel('Trade Number')
plt.ylabel('Cumulative Profit (pips)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.show()

## 4. Test Multiple Grid Configurations

Let's test different grid sizes and level counts to find the optimal configuration.

In [ ]:
# Test multiple configurations
grid_sizes = [10, 20, 30, 50]
num_levels_list = [5, 10, 15, 20]

results = []
for grid_size in grid_sizes:
    for num_levels in num_levels_list:
        result = grid_sim.evaluate_pair(i_pair, grid_size, num_levels, price_data)
        if result is not None:
            results.append(result)

print(f"Tested {len(results)} configurations")

In [ ]:
# Create summary DataFrame
results_list = [r.result_ob() for r in results]
results_df = pd.DataFrame.from_dict(results_list)

# Sort by total gain
results_df_sorted = results_df.sort_values('total_gain', ascending=False)

print("Top 10 Configurations:")
results_df_sorted.head(10)

## 5. Visualize Performance Across Configurations

In [ ]:
# Heatmap of performance by grid size and num levels
pivot_table = results_df.pivot_table(
    values='total_gain', 
    index='num_levels', 
    columns='grid_size_pips'
)

plt.figure(figsize=(10, 6))
plt.imshow(pivot_table.values, cmap='RdYlGn', aspect='auto')
plt.colorbar(label='Total Gain (pips)')
plt.xticks(range(len(pivot_table.columns)), pivot_table.columns)
plt.yticks(range(len(pivot_table.index)), pivot_table.index)
plt.xlabel('Grid Size (pips)')
plt.ylabel('Number of Levels')
plt.title(f'Grid Strategy Performance Heatmap - {pairname}')
plt.show()

In [ ]:
# Compare win rates across configurations
plt.figure(figsize=(12, 6))
plt.scatter(results_df.num_trades, results_df.total_gain, 
            c=results_df.win_rate, s=100, cmap='viridis', alpha=0.6)
plt.colorbar(label='Win Rate (%)')
plt.xlabel('Number of Trades')
plt.ylabel('Total Gain (pips)')
plt.title('Trades vs Profit (colored by Win Rate)')
plt.grid(True, alpha=0.3)
plt.show()

## 6. Run Full Backtest on Multiple Pairs

To run the full backtest on multiple currency pairs, uncomment and run the cell below.

**Note:** This may take several minutes depending on the amount of historical data.

In [ ]:
# Uncomment to run full backtest
# grid_sim.run()

## 7. Analyze Results from Full Backtest

After running the full backtest, you can load and analyze the results.

In [ ]:
# Load results from full backtest
try:
    full_results = grid_result.load_results('grid_test_res.pkl')
    print(f"Loaded {len(full_results)} results")
    
    # Overall analysis
    analysis = grid_result.analyze_results(full_results)
    print("\nOverall Analysis:")
    for key, value in analysis.items():
        print(f"  {key}: {value}")
    
    # Top configurations
    print("\nTop 10 Configurations Across All Pairs:")
    top_configs = grid_result.get_best_configurations(full_results, top_n=10)
    display(top_configs)
    
except FileNotFoundError:
    print("No results file found. Run grid_sim.run() first.")

In [ ]:
# Load all trades
try:
    all_trades = grid_result.load_all_trades('grid_all_trades.pkl')
    print(f"Total trades across all pairs: {len(all_trades)}")
    
    # Analyze by pair
    print("\nPerformance by Currency Pair:")
    pair_performance = all_trades.groupby('PAIR').agg({
        'GAIN': ['count', 'sum', 'mean'],
        'DURATION': 'mean'
    }).round(2)
    pair_performance.columns = ['Trades', 'Total Pips', 'Avg Pips', 'Avg Duration (hrs)']
    pair_performance = pair_performance.sort_values('Total Pips', ascending=False)
    display(pair_performance)
    
except FileNotFoundError:
    print("No trades file found. Run grid_sim.run() first.")

## 8. Compare Grid Strategy with Moving Average Strategy

If you have MA strategy results, you can compare them here.

In [ ]:
# Compare strategies
try:
    ma_results = pd.read_pickle('ma_test_res.pkl')
    grid_results_full = pd.read_pickle('grid_test_res.pkl')
    
    print("Strategy Comparison:")
    print(f"\nMoving Average Strategy:")
    print(f"  Total Trades: {ma_results.num_trades.sum()}")
    print(f"  Total Gain: {ma_results.total_gain.sum():.2f} pips")
    print(f"  Avg Gain per Trade: {ma_results.total_gain.sum() / ma_results.num_trades.sum():.2f} pips")
    
    print(f"\nGrid Trading Strategy:")
    print(f"  Total Trades: {grid_results_full.num_trades.sum()}")
    print(f"  Total Gain: {grid_results_full.total_gain.sum():.2f} pips")
    print(f"  Avg Gain per Trade: {grid_results_full.total_gain.sum() / grid_results_full.num_trades.sum():.2f} pips")
    
except FileNotFoundError as e:
    print(f"Could not load comparison data: {e}")

## Conclusions

Grid trading is most effective in:
- **Ranging markets** with clear support/resistance levels
- **Low volatility** periods
- Currency pairs that tend to mean-revert

Key considerations:
- Grid size should match the pair's typical price movement
- More levels = more trades but also more risk exposure
- Monitor for trending markets where grid strategies can underperform
- Consider combining with trend filters or volatility measures